Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
import numpy as np
import time
import copy
import math
from typing import Callable
import matplotlib.pyplot as plt


In [2]:
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [3]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [4]:
CONSTRAINTS

array([[ 95,   6],
       [303, 319],
       [290, 175]])

## TEST PROBLEMS

In [5]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [6]:
from dataclasses import dataclass


@dataclass
class Solution:
    knapsacks: list[set[int]]
    total_value: int

    def get_items_left(self) -> set[int]:
        items_left = set(range(NUM_ITEMS))
        for ks in self.knapsacks:
            items_left -= ks
        return items_left

In [7]:
def find_first_solution(
    knapsacks: list[set[int]] = None,
) -> Solution:
    if knapsacks is None:
        knapsacks = [set() for _ in range(NUM_KNAPSACKS)]

    running_totals = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS), dtype=int)
    total_value = 0

    # We build a simple probability distribution that favors items with higher value-to-weight ratios.
    nudged_probs = np.array(
        [
            1e-3 + np.log(VALUES[i] + 1) - (np.log((WEIGHTS[i] + 1)).sum())
            for i in range(NUM_ITEMS)
        ]
    )

    free_items = set(range(NUM_ITEMS))
    for ks in knapsacks:
        free_items -= ks

    while len(free_items) > 0:
        probs = nudged_probs[list(free_items)]
        probs = probs / probs.sum()
        item = rng.choice(list(free_items), p=probs)

        # Calculate availability after adding the item to each knapsack
        availability_after = CONSTRAINTS - (running_totals + WEIGHTS[item])
        # print((availability_after / CONSTRAINTS).shape)
        # Find knapsacks that can actually accommodate the item
        available_knapsacks = np.where((availability_after >= 0).all(axis=1))[0]
        # print(available_knapsacks)
        # If no knapsack can accommodate the item, skip it
        if len(available_knapsacks) == 0:
            # This item cannot be added to any knapsack (we may be in a local optimum)
            free_items.remove(item)
            continue

        if len(available_knapsacks) == 1:
            knapsack_i = available_knapsacks[0]

        else:
            balance_after = (
                CONSTRAINTS[available_knapsacks]
                - availability_after[available_knapsacks]
            )
            balance_after = balance_after.max(axis=1) - balance_after.min(axis=1)
            nudged_probs2 = 1 / (balance_after + 4)
            nudged_probs2 = nudged_probs2 / nudged_probs2.sum()
            # print(balance_after, balance_after.shape)
            # Select the knapsack that will have the most balanced load between the dimensions of constraint after adding the item
            knapsack_i = available_knapsacks[balance_after.argmin()]
            # knapsack_i = rng.choice(available_knapsacks, p=nudged_probs2)

        knapsacks[knapsack_i].add(item)
        free_items.remove(item)
        running_totals[knapsack_i] += WEIGHTS[item]
        total_value += VALUES[item]

    return Solution(knapsacks=knapsacks, total_value=total_value)

In [8]:
first_solution = find_first_solution()
print(first_solution.total_value)

1020


In [9]:
def benchmark_solution(
    solution_factory: Callable[[], Solution], n_runs: int = 10
) -> Solution:
    start_time = time.time()

    runs_points = np.zeros(n_runs)
    best_knapsacks: list[set[int]] = []
    best_value = 0

    for run_i in range(n_runs):
        solution = solution_factory()
        total_value = solution.total_value

        runs_points[run_i] = total_value
        if total_value > best_value:
            best_value = total_value
            best_knapsacks = solution.knapsacks

    end_time = time.time()
    avg_time = (end_time - start_time) / n_runs

    print("Best total value found:", max(runs_points))
    # print("Best knapsacks found:", best_knapsacks)

    print(f"Statistics over runs (n = {n_runs}):")
    print(f"  Min: {runs_points.min():.2f}")
    print(f"  Max: {runs_points.max():.2f}")
    print(f"  Mean: {runs_points.mean():.2f}")
    print(f"  Std: {runs_points.std():.2f}")
    print(f"Elapsed time: {(avg_time):.4f} seconds")

    return Solution(knapsacks=best_knapsacks, total_value=best_value)

In [10]:
benchmarked_solution = benchmark_solution(find_first_solution, n_runs=10)

Best total value found: 1065.0
Statistics over runs (n = 10):
  Min: 926.00
  Max: 1065.00
  Mean: 1001.70
  Std: 45.98
Elapsed time: 0.0011 seconds


In [11]:
def tweak_solution(original: Solution) -> Solution:
    # Shallow copy knapsacks (sets) and build an item->knapsack map for O(1) lookups
    new_knapsacks = [set(s) for s in original.knapsacks]
    item_to_knapsack = np.full(NUM_ITEMS, -1, dtype=int)
    for k, ks in enumerate(new_knapsacks):
        for it in ks:
            item_to_knapsack[it] = k

    # Compute running_totals efficiently per knapsack
    running_totals = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS), dtype=int)
    for k, ks in enumerate(new_knapsacks):
        if ks:
            running_totals[k] = WEIGHTS[list(ks)].sum(axis=0)

    total_value = int(original.total_value)

    # Decide how many items to tweak
    n_tweaks = int(rng.integers(1, max(2, NUM_ITEMS // 20)))

    for _ in range(n_tweaks):
        item = int(rng.integers(0, NUM_ITEMS))
        current = int(item_to_knapsack[item])

        # Remove from current knapsack if present (update map and totals incrementally)
        if current != -1:
            new_knapsacks[current].remove(item)
            running_totals[current] -= WEIGHTS[item]
            total_value -= int(VALUES[item])
            item_to_knapsack[item] = -1

        # Find candidate knapsacks that can accept the item (vectorized check)
        # compute needed = running_totals + WEIGHTS[item] and compare to CONSTRAINTS
        needed = running_totals + WEIGHTS[item]  # shape (K, D)
        feasible = np.all(needed <= CONSTRAINTS, axis=1)
        # exclude current (already removed) from candidates
        feasible_indices = np.where(feasible)[0].tolist()
        if current in feasible_indices:
            feasible_indices.remove(current)

        if feasible_indices:
            chosen = int(rng.choice(feasible_indices))
            new_knapsacks[chosen].add(item)
            running_totals[chosen] += WEIGHTS[item]
            total_value += int(VALUES[item])
            item_to_knapsack[item] = chosen

    return Solution(knapsacks=new_knapsacks, total_value=total_value)


In [12]:
tweak_solution(benchmarked_solution).total_value

1013

In [13]:
def benchmark_tweak(
    original: Solution,
    tweak_function: Callable[[Solution], Solution],
    n_runs: int = 10,
) -> Solution:
    start_time = time.time()

    runs_points = np.zeros(n_runs)
    best_solution = original

    for run_i in range(n_runs):
        solution = tweak_function(original)

        runs_points[run_i] = solution.total_value
        if solution.total_value > best_solution.total_value:
            best_solution = solution

    end_time = time.time()
    avg_time = (end_time - start_time) / n_runs

    print("Best total value found after tweak:", max(runs_points))
    # print("Best knapsacks found:", best_knapsacks)

    print(f"Statistics over tweak runs (n = {n_runs}):")
    print(f"  Min: {runs_points.min():.2f}")
    print(f"  Max: {runs_points.max():.2f}")
    print(f"  Mean: {runs_points.mean():.2f}")
    print(f"  Std: {runs_points.std():.2f}")
    print(f"Elapsed time: {(avg_time):.4f} seconds")

    return best_solution


In [14]:
tweaked_solution = benchmark_tweak(benchmarked_solution, tweak_solution, n_runs=10)

Best total value found after tweak: 1053.0
Statistics over tweak runs (n = 10):
  Min: 980.00
  Max: 1053.00
  Mean: 1013.90
  Std: 18.64
Elapsed time: 0.0002 seconds


In [15]:
tweaked_solution = benchmark_tweak(tweaked_solution, tweak_solution, n_runs=100)

Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1022.20
  Std: 31.10
Elapsed time: 0.0001 seconds


In [16]:
from tqdm import tqdm

In [ ]:
def plot_fitness_history(fitness_history, title="Fitness Score Over Iterations"):
    """
    Plot the fitness score history over iterations.
    
    Args:
        fitness_history: List of fitness scores (total_value) over iterations
        title: Title for the plot
    """
    plt.figure(figsize=(10, 6))
    plt.plot(fitness_history, marker='o', linestyle='-', linewidth=2, markersize=4)
    plt.xlabel('Iteration', fontsize=12)
    plt.ylabel('Fitness Score (Total Value)', fontsize=12)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"\nFitness Statistics:")
    print(f"  Initial: {fitness_history[0]:.2f}")
    print(f"  Final: {fitness_history[-1]:.2f}")
    print(f"  Improvement: {fitness_history[-1] - fitness_history[0]:.2f}")
    print(f"  Best: {max(fitness_history):.2f}")


In [17]:
# Optimization loop with fitness tracking and visualization
fitness_history = [tweaked_solution.total_value]

for iteration in tqdm(range(10)):
    tweaked_solution = benchmark_tweak(tweaked_solution, tweak_solution, n_runs=100, return_history=False)
    fitness_history.append(tweaked_solution.total_value)

# Plot the fitness history
plot_fitness_history(fitness_history, "Fitness Score Over Optimization Iterations")


100%|██████████| 10/10 [00:00<00:00, 272.87it/s]

Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1018.46
  Std: 32.58
Elapsed time: 0.0001 seconds
Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1019.09
  Std: 31.83
Elapsed time: 0.0000 seconds
Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1025.19
  Std: 29.18
Elapsed time: 0.0000 seconds
Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1019.78
  Std: 32.42
Elapsed time: 0.0000 seconds
Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1025.07
  Std: 29.65
Elapsed time: 0.0000 seconds
Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1024.47
  Std: 33.22
Elapsed time: 0.0000 

## TEST PROBLEMS

In [18]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [19]:
first_solution = benchmark_solution(find_first_solution, n_runs=100)
first_solution.total_value
tweaked_solution = first_solution

# Track fitness for Problem 1
fitness_history = [tweaked_solution.total_value]
for iteration in tqdm(range(10)):
    tweaked_solution = benchmark_tweak(tweaked_solution, tweak_solution, n_runs=100)
    fitness_history.append(tweaked_solution.total_value)

print(tweaked_solution.total_value)

# Visualize Problem 1 results
plot_fitness_history(fitness_history, "Problem 1: Fitness Score Over Iterations")


Best total value found: 1065.0
Statistics over runs (n = 100):
  Min: 843.00
  Max: 1065.00
  Mean: 997.33
  Std: 54.76
Elapsed time: 0.0010 seconds


  0%|          | 0/10 [00:00<?, ?it/s]

Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1022.66
  Std: 29.35
Elapsed time: 0.0000 seconds
Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1022.72
  Std: 32.61
Elapsed time: 0.0001 seconds
Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1023.22
  Std: 32.54
Elapsed time: 0.0000 seconds
Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1017.91
  Std: 34.86
Elapsed time: 0.0001 seconds
Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1024.25
  Std: 33.03
Elapsed time: 0.0001 seconds
Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1025.08
  Std: 32.68
Elapsed time: 0.0000 

100%|██████████| 10/10 [00:00<00:00, 200.05it/s]

Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1021.06
  Std: 32.07
Elapsed time: 0.0000 seconds
1065


In [20]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [21]:
first_solution = benchmark_solution(find_first_solution, n_runs=1000)
first_solution.total_value
tweaked_solution = first_solution

# Track fitness for Problem 2
fitness_history = [tweaked_solution.total_value]
for iteration in tqdm(range(250)):
    tweaked_solution = benchmark_tweak(tweaked_solution, tweak_solution, n_runs=250)
    fitness_history.append(tweaked_solution.total_value)

print(tweaked_solution.total_value)

# Visualize Problem 2 results
plot_fitness_history(fitness_history, "Problem 2: Fitness Score Over Iterations")


Best total value found: 39355.0
Statistics over runs (n = 1000):
  Min: 27817.00
  Max: 39355.00
  Mean: 34308.07
  Std: 1755.80
Elapsed time: 0.0028 seconds


  3%|▎         | 7/250 [00:00<00:03, 62.09it/s]

Best total value found after tweak: 40004.0
Statistics over tweak runs (n = 250):
  Min: 37041.00
  Max: 40004.00
  Mean: 38694.06
  Std: 498.42
Elapsed time: 0.0001 seconds
Best total value found after tweak: 40922.0
Statistics over tweak runs (n = 250):
  Min: 37479.00
  Max: 40922.00
  Mean: 39355.14
  Std: 618.43
Elapsed time: 0.0001 seconds
Best total value found after tweak: 41872.0
Statistics over tweak runs (n = 250):
  Min: 38586.00
  Max: 41872.00
  Mean: 40355.85
  Std: 682.64
Elapsed time: 0.0001 seconds
Best total value found after tweak: 42189.0
Statistics over tweak runs (n = 250):
  Min: 38992.00
  Max: 42189.00
  Mean: 40987.52
  Std: 617.38
Elapsed time: 0.0000 seconds
Best total value found after tweak: 42189.0
Statistics over tweak runs (n = 250):
  Min: 39774.00
  Max: 42189.00
  Mean: 41292.75
  Std: 582.99
Elapsed time: 0.0001 seconds
Best total value found after tweak: 42429.0
Statistics over tweak runs (n = 250):
  Min: 38705.00
  Max: 42429.00
  Mean: 41340.13

  9%|▉         | 23/250 [00:00<00:03, 71.03it/s]

Best total value found after tweak: 45590.0
Statistics over tweak runs (n = 250):
  Min: 43042.00
  Max: 45590.00
  Mean: 44701.03
  Std: 610.63
Elapsed time: 0.0001 seconds
Best total value found after tweak: 45590.0
Statistics over tweak runs (n = 250):
  Min: 42795.00
  Max: 45590.00
  Mean: 44675.74
  Std: 599.55
Elapsed time: 0.0001 seconds
Best total value found after tweak: 45590.0
Statistics over tweak runs (n = 250):
  Min: 42126.00
  Max: 45590.00
  Mean: 44650.50
  Std: 629.68
Elapsed time: 0.0001 seconds
Best total value found after tweak: 45590.0
Statistics over tweak runs (n = 250):
  Min: 42792.00
  Max: 45590.00
  Mean: 44704.87
  Std: 587.56
Elapsed time: 0.0001 seconds
Best total value found after tweak: 45590.0
Statistics over tweak runs (n = 250):
  Min: 43205.00
  Max: 45590.00
  Mean: 44753.46
  Std: 563.35
Elapsed time: 0.0001 seconds
Best total value found after tweak: 45771.0
Statistics over tweak runs (n = 250):
  Min: 42232.00
  Max: 45771.00
  Mean: 44747.66

...

Best total value found after tweak: 48401.0
Statistics over tweak runs (n = 250):
  Min: 45550.00
  Max: 48401.00
  Mean: 47373.37
  Std: 641.96
Elapsed time: 0.0001 seconds
Best total value found after tweak: 48401.0
Statistics over tweak runs (n = 250):
  Min: 45480.00
  Max: 48401.00
  Mean: 47424.59
  Std: 625.39
Elapsed time: 0.0001 seconds
Best total value found after tweak: 48401.0
Statistics over tweak runs (n = 250):
  Min: 45667.00
  Max: 48401.00
  Mean: 47391.23
  Std: 583.48
Elapsed time: 0.0001 seconds
Best total value found after tweak: 48401.0
Statistics over tweak runs (n = 250):
  Min: 45883.00
  Max: 48401.00
  Mean: 47348.66
  Std: 623.94
Elapsed time: 0.0001 seconds
Best total value found after tweak: 48401.0
Statistics over tweak runs (n = 250):
  Min: 45801.00
  Max: 48401.00
  Mean: 47432.24
  Std: 533.02
Elapsed time: 0.0001 seconds
Best total value found after tweak: 48401.0
Statistics over tweak runs (n = 250):
  Min: 45635.00
  Max: 48401.00
  Mean: 47428.20

 96%|█████████▋| 241/250 [00:05<00:00, 60.10it/s]

Best total value found after tweak: 48401.0
Statistics over tweak runs (n = 250):
  Min: 45755.00
  Max: 48401.00
  Mean: 47389.33
  Std: 631.83
Elapsed time: 0.0001 seconds
Best total value found after tweak: 48401.0
Statistics over tweak runs (n = 250):
  Min: 45487.00
  Max: 48401.00
  Mean: 47467.96
  Std: 613.46
Elapsed time: 0.0001 seconds
Best total value found after tweak: 48401.0
Statistics over tweak runs (n = 250):
  Min: 45647.00
  Max: 48401.00
  Mean: 47401.07
  Std: 651.39
Elapsed time: 0.0002 seconds
Best total value found after tweak: 48401.0
Statistics over tweak runs (n = 250):
  Min: 45834.00
  Max: 48401.00
  Mean: 47419.83
  Std: 595.53
Elapsed time: 0.0001 seconds
Best total value found after tweak: 48401.0
Statistics over tweak runs (n = 250):
  Min: 45396.00
  Max: 48401.00
  Mean: 47411.84
  Std: 632.25
Elapsed time: 0.0001 seconds
Best total value found after tweak: 48401.0
Statistics over tweak runs (n = 250):
  Min: 45136.00
  Max: 48401.00
  Mean: 47396.32

100%|██████████| 250/250 [00:05<00:00, 45.34it/s]

Best total value found after tweak: 48401.0
Statistics over tweak runs (n = 250):
  Min: 45300.00
  Max: 48401.00
  Mean: 47381.55
  Std: 633.47
Elapsed time: 0.0001 seconds
Best total value found after tweak: 48401.0
Statistics over tweak runs (n = 250):
  Min: 45168.00
  Max: 48401.00
  Mean: 47406.69
  Std: 608.99
Elapsed time: 0.0001 seconds
Best total value found after tweak: 48401.0
Statistics over tweak runs (n = 250):
  Min: 45534.00
  Max: 48401.00
  Mean: 47389.62
  Std: 619.50
Elapsed time: 0.0001 seconds
Best total value found after tweak: 48401.0
Statistics over tweak runs (n = 250):
  Min: 45345.00
  Max: 48401.00
  Mean: 47435.62
  Std: 572.56
Elapsed time: 0.0000 seconds
Best total value found after tweak: 48401.0
Statistics over tweak runs (n = 250):
  Min: 45600.00
  Max: 48401.00
  Mean: 47375.30
  Std: 648.38
Elapsed time: 0.0000 seconds
Best total value found after tweak: 48401.0
Statistics over tweak runs (n = 250):
  Min: 45678.00
  Max: 48401.00
  Mean: 47455.69

In [22]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10,
    1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS,
    size=(NUM_KNAPSACKS, NUM_DIMENSIONS),
)

In [ ]:
first_solution = benchmark_solution(find_first_solution, n_runs=10)
first_solution.total_value
tweaked_solution = first_solution

# Track fitness for Problem 3
fitness_history = [tweaked_solution.total_value]
for iteration in tqdm(range(100)):
    tweaked_solution = benchmark_tweak(tweaked_solution, tweak_solution, n_runs=50)
    fitness_history.append(tweaked_solution.total_value)

print(tweaked_solution.total_value)

# Visualize Problem 3 results
plot_fitness_history(fitness_history, "Problem 3: Fitness Score Over Iterations")


Best total value found: 1227842.0
Statistics over runs (n = 10):
  Min: 1188740.00
  Max: 1227842.00
  Mean: 1210450.80
  Std: 11438.30
Elapsed time: 1.1246 seconds


  2%|▏         | 1/50 [00:00<00:35,  1.38it/s]

Best total value found after tweak: 1248100.0
Statistics over tweak runs (n = 250):
  Min: 1220422.00
  Max: 1248100.00
  Mean: 1230273.30
  Std: 5764.97
Elapsed time: 0.0029 seconds


  4%|▍         | 2/50 [00:01<00:34,  1.39it/s]

Best total value found after tweak: 1276074.0
Statistics over tweak runs (n = 250):
  Min: 1245613.00
  Max: 1276074.00
  Mean: 1258881.38
  Std: 7132.27
Elapsed time: 0.0029 seconds


...

Best total value found after tweak: 1696567.0
Statistics over tweak runs (n = 250):
  Min: 1674754.00
  Max: 1696567.00
  Mean: 1687373.11
  Std: 4015.46
Elapsed time: 0.0032 seconds


 98%|█████████▊| 49/50 [00:40<00:00,  1.18it/s]

Best total value found after tweak: 1699727.0
Statistics over tweak runs (n = 250):
  Min: 1678986.00
  Max: 1699727.00
  Mean: 1690764.41
  Std: 3870.49
Elapsed time: 0.0038 seconds


100%|██████████| 50/50 [00:41<00:00,  1.20it/s]

Best total value found after tweak: 1702294.0
Statistics over tweak runs (n = 250):
  Min: 1682109.00
  Max: 1702294.00
  Mean: 1695627.03
  Std: 3830.26
Elapsed time: 0.0035 seconds
1702294
